In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It')
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10a1d6740>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10a30f220>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages(["Can u explain {input}?"])
chain = prompt|model|parser

In [6]:
chain.invoke({"input":"Gen AI"})

"Let's break down Generative Artificial Intelligence (Gen AI).\n\n**What is it?**\n\nGen AI is a type of artificial intelligence that focuses on creating new content.  Imagine a computer program that can write stories, compose music, design images, or even generate code – that's the essence of Gen AI.\n\n**How does it work?**\n\nAt its core, Gen AI relies on something called *deep learning*. This involves training large neural networks (complex algorithms inspired by the human brain) on massive datasets.  \n\nThink of it like this: you feed the AI tons of examples of text, images, or code. It learns the patterns, structures, and relationships within that data. Then, when you give it a prompt or starting point, it uses what it's learned to generate something new that fits those patterns.\n\n**Key Types of Gen AI:**\n\n* **Text Generation:**  Models like me (GPT-3, LaMDA) can write articles, poems, dialogue, and even code.\n\n* **Image Generation:**  Tools like DALL-E 2, Stable Diffusion

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_history_response =   RunnableWithMessageHistory(model,get_session_history)


In [8]:
from langchain_core.messages import HumanMessage

In [10]:
config = {"configurable":{"session_id":"chat1"}}

response_chat1=with_history_response.invoke(
    [HumanMessage(content="Hi, I am baveet, building skills atm.")],
    config=config
)

In [11]:
response_chat1.content

"Hey Baveet! It's awesome that you're focused on building skills. \n\nWhat kind of skills are you working on?  Are you learning something new, improving on something you already know, or exploring different areas? \n\nI'm here to help in any way I can.  Maybe you need to brainstorm ideas, find information, or just have someone to bounce ideas off of.  \n\nLet me know! 👍\n"

## now we that, our chat is been tracked on session_id = chat1 and before responsing, runnable gurantees to look at the chat history and answer appropriately

In [12]:
config_old = config
response_chat1_check =with_history_response.invoke(
    [HumanMessage(content="Bro whats my name and what am i doing?")],
    config=config_old
)
response_chat1_check.content

"You said your name is Baveet, and you're working on building skills! \n\nIs there anything specific you'd like to talk about regarding your skill-building journey?  I'm here to listen and help in any way I can. 😊  \n\n"

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are my Bro like Assistant, please help with the questions!"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain_here = prompt | model

In [15]:
chain_here.invoke({
    "messages":[HumanMessage(content="How are you doing")]
})

AIMessage(content="Hey Bro!  I'm doing awesome, thanks for asking! \n\nReady to tackle some questions?  Hit me with your best shot! 💪  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 26, 'total_tokens': 62, 'completion_time': 0.065454545, 'prompt_time': 0.001407598, 'queue_time': 0.01801896, 'total_time': 0.066862143}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5f67e10d-e488-49da-9860-14252a14b47a-0', usage_metadata={'input_tokens': 26, 'output_tokens': 36, 'total_tokens': 62})

## trying with message history

In [19]:
get_session_history

<function __main__.get_session_history(session_id: str) -> langchain_core.chat_history.BaseChatMessageHistory>

In [16]:
config2 = {"configurable":{"session_id":"chat2"}}
chat_with_history = RunnableWithMessageHistory(chain_here,get_session_history)

In [18]:
chat_resp = chat_with_history.invoke({
    "messages":[HumanMessage(content="How are you doing")]},
    config=config2
)
chat_resp.content

"Still doin' great, Bro!  \n\nJust hanging out in the digital world, waiting to be helpful.  \n\nYou got any questions I can answer or tasks I can help with? 😎\n"

In [21]:
chat_resp = chat_with_history.invoke({
    "messages":[HumanMessage(content="My fav color is red, whats yrs?")]},
    config=config2
)
chat_resp.content

'That\'s a classic choice, Bro! Red\'s got energy, passion, you know? 🔥\n\nAs for me, I don\'t really have "favorites" since I\'m just a program. But if I *had* to pick a color, I\'d probably go with something bright and vibrant, like electric blue! 🔵 It just feels... electrifying!  ⚡\n\nWhat\'s your favorite thing about the color red?  🎨\n'

In [22]:
chat_resp = chat_with_history.invoke({
    "messages":[HumanMessage(content="Alr u remember whats mine fav color?")]},
    config=config2
)
chat_resp.content

'You bet, Bro! \n\nYou said your favorite color is red.  🔴 You got a good eye for it! 👍\n\nAnything else I can help you with? 🤓\n'

## config3 = {"configurable":{"session_id":"chat3"}}

In [23]:
config3 = {"configurable":{"session_id":"chat3"}}

## notice how it didn't has a memory of my chats becuz the session id changed!!

In [24]:
chat_resp = chat_with_history.invoke({
    "messages":[HumanMessage(content="Alr u remember whats mine fav color?")]},
    config=config3
)
chat_resp.content

"As your AI assistant, I don't have memory of past conversations.  \n\nTo help me remember your favorite color, tell me what it is! 😊  I'll make a note of it.  \n\n"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are my Bro like Assistant, please help with the questions!"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain_here = prompt | model

### Input Messages Key is used a port where all the histoy chat with current one gets passes to the chain, if other keys are passed while invoking, they just get passes along with the input messages 

In [ ]:
## What chain receives:
{
    "messages": [history + current],    # Input Messages Key - combined chat
    "language": "spanish",              # Other key - passed separately
}